# [Baseball Databank](http://www.seanlahman.com/baseball-archive/statistics/)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from pathlib import Path
from typing import Tuple, Callable

In [3]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

In [4]:
# show matplotlib plots in notebook
%matplotlib inline

In [5]:
# set matplotlib style/theme
plt.style.use("fivethirtyeight")

In [6]:
# plotly defaults
pio.renderers.default = "notebook"
pio.templates.default = "seaborn"

In [7]:
from tabulate import tabulate
from IPython.core.display import display, Markdown, HTML

def markdown(s: str):
    display(Markdown(s))
    
def html(s: str):
    display(HTML(s))

In [8]:
# path to data files
datapath = Path("../baseballdatabank/core")

___________________
### Batting info

In [9]:
batting_all = pd.read_csv(datapath/"batting.csv")

### People info

In [10]:
people = pd.read_csv(datapath/"people.csv")
for col in "debut", "finalGame":
    people[col] = pd.to_datetime(people[col])

In [11]:
# slimmed people dataframe with player ID, full name, first and final games as datetimes
people_slim = pd.concat([people["playerID"], 
                         people[["nameFirst", "nameLast"]].fillna("").agg(" ".join, axis=1).rename("fullName"), 
                         people["debut"].apply(lambda dt: dt.year).fillna(0).astype(np.int), 
                         people["finalGame"].apply(lambda dt: dt.year).fillna(0).astype(np.int)], 
                        axis=1)

_____________________________________________

# Team stats

In [17]:
teams = pd.read_csv(datapath/"teams.csv")

In [20]:
teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2955 entries, 0 to 2954
Data columns (total 48 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   yearID          2955 non-null   int64  
 1   lgID            2905 non-null   object 
 2   teamID          2955 non-null   object 
 3   franchID        2955 non-null   object 
 4   divID           1438 non-null   object 
 5   Rank            2955 non-null   int64  
 6   G               2955 non-null   int64  
 7   Ghome           2556 non-null   float64
 8   W               2955 non-null   int64  
 9   L               2955 non-null   int64  
 10  DivWin          1410 non-null   object 
 11  WCWin           774 non-null    object 
 12  LgWin           2927 non-null   object 
 13  WSWin           2598 non-null   object 
 14  R               2955 non-null   int64  
 15  AB              2955 non-null   int64  
 16  H               2955 non-null   int64  
 17  2B              2955 non-null   i

# Franchise success
___________________
### W/L records

In [13]:
# translate franchID to team name(s)
def franch_to_name(franchID: str):
    return "/".join(teams[teams["franchID"] == franchID]["name"].unique())

In [987]:
teams["Win %"] = teams["W"] / teams["G"]

In [996]:
# best record in any season
best = teams.loc[teams["Win %"].idxmax()][["franchID", "yearID", "G", "W", "Win %"]]

In [997]:
# best record in a 160+ game season
df = teams[teams["G"] >= 160]
best_160 = df.loc[df["Win %"].idxmax()][["franchID", "yearID", "G", "W", "Win %"]]

In [998]:
# worst record in any season
worst = teams.loc[teams["Win %"].idxmin()][["franchID", "yearID", "G", "W", "Win %"]]

In [999]:
# worst record in a 160+ game season
df = teams[teams["G"] >= 160]
worst_160 = df.loc[df["Win %"].idxmin()][["franchID", "yearID", "G", "W", "Win %"]]

In [1000]:
# most wins in any season
most_wins = df.loc[df["W"].idxmax()][["franchID", "yearID","G", "W", "Win %"]]

In [1015]:
# stitch together the various records, rename for readable columns, translate franchise ID to team name(s)
franchise_records = pd.DataFrame([most_wins, best, best_160, worst, worst_160], 
                                 index=["Most wins",
                                        "Best record", 
                                        "Best record (160+ games)",
                                        "Worst Record",
                                        "Worst Record (160+ games)"]
                                ).rename({"franchID": "team",
                                          "yearID": "year",
                                          "W": "wins",
                                          "G": "games"}, axis=1)
                                        
franchise_records["team"] = franchise_records["team"].apply(franch_to_name)
franchise_records

,team,year,games,wins,Win %
Most wins,Seattle Mariners,2001,162,116,0.716049
Best record,Boston Red Stockings,1875,82,71,0.865854
Best record (160+ games),Seattle Mariners,2001,162,116,0.716049
Worst Record,Washington Nationals,1872,11,0,0.000000
Worst Record (160+ games),New York Mets,1962,161,40,0.248447


--------------------------

### Most winning seasons
In their current state, these are probably more indicative of franchise longevity than overall success.
Percentage of seasons  with a winning record might be useful, though it likely overlaps somewhat with 
the average winning percentage calculated below

In [1038]:
# W/L per team per year
win_pct = teams.pivot_table(index="franchID", columns="yearID", values="Win %")
win_pct_20s = win_pct.dropna(thresh=20)

In [1097]:
def most_wins(df: pd.DataFrame) -> Tuple[str, int]:
    seasons = (df > 0.500).sum(axis=1)
    team = seasons.idxmax()
    return(team, seasons[team])

def fewest_wins(df: pd.DataFrame) -> Tuple[str, int]:
    seasons = (df > 0.500).sum(axis=1)
    team = seasons.idxmin()
    return(team, seasons[team])

In [1099]:
team_h_any, high_any = most_wins(win_pct)
team_h_20, high_20 = most_wins(win_pct_20s)
team_l_any, low_any = fewest_wins(win_pct)
team_l_20, low_20 = fewest_wins(win_pct_20s)

In [1100]:
html(tabulate([["max (overall)", high_any, franch_to_name(team_h_any)],
               ["max (20+ seasons)", high_20, franch_to_name(team_h_20)],
               ["min (overall)", low_any, franch_to_name(team_l_any)],
               ["min (20+ seasons)", low_20, franch_to_name(team_l_20)],

              ], 
              tablefmt="html", 
              headers=["Category", "Winning Seasons", "Franchise"],
              stralign="center", numalign="center"))

Category,Winning Seasons,Franchise
max (overall),97,Baltimore Orioles/New York Highlanders/New York Yankees
max (20+ seasons),97,Baltimore Orioles/New York Highlanders/New York Yankees
min (overall),0,Altoona Mountain City
min (20+ seasons),7,Florida Marlins/Miami Marlins


#### Highest average winning percentage

In [1123]:
df = (win_pct > 0.500).mean(axis=1)
team = df.idxmax()
num_seasons = teams.loc[teams["franchID"] == team].shape[0]
markdown(f"The **{franch_to_name(team)}** franchise has the highest average winning percentage"
         f" of **{df[team]:.3f}** over {num_seasons} season(s)")

The **Baltimore Orioles/New York Highlanders/New York Yankees** franchise has the highest average winning percentage of **0.647** over 120 season(s)

#### Lowest average winning percentage

In [1124]:
df = (win_pct > 0.500).mean(axis=1)
team = df.idxmin()
num_seasons = teams.loc[teams["franchID"] == team].shape[0]
markdown(f"The **{franch_to_name(team)}** franchise has the lowest average winning percentage"
         f" of **{df[team]:.3f}** over {num_seasons} season(s)")

The **Altoona Mountain City** franchise has the lowest average winning percentage of **0.000** over 1 season(s)

#### Current teams (1998+) win percentages

In [1126]:
teams_1998_wpct = teams.loc[teams["yearID"] >= 1998].pivot_table(index="franchID", columns="yearID", values="Win %")
teams_1998_wpct

yearID,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
franchID,,,,,,,,,,,,,,,,,,,,,
ANA,0.524691,0.432099,0.506173,0.462963,0.611111,0.475309,0.567901,0.586420,0.549383,0.580247,...,0.530864,0.549383,0.481481,0.604938,0.524691,0.456790,0.493827,0.493827,0.444444,0.433333
ARI,0.401235,0.617284,0.524691,0.567901,0.604938,0.518519,0.314815,0.475309,0.469136,0.555556,...,0.580247,0.500000,0.500000,0.395062,0.487654,0.425926,0.574074,0.506173,0.524691,0.416667
ATL,0.654321,0.635802,0.586420,0.543210,0.627329,0.623457,0.592593,0.555556,0.487654,0.518519,...,0.549383,0.580247,0.592593,0.487654,0.413580,0.422360,0.444444,0.555556,0.598765,0.583333
BAL,0.487654,0.481481,0.456790,0.388889,0.413580,0.435583,0.481481,0.456790,0.432099,0.425926,...,0.425926,0.574074,0.524691,0.592593,0.500000,0.549383,0.462963,0.290123,0.333333,0.416667
BOS,0.567901,0.580247,0.524691,0.509317,0.574074,0.586420,0.604938,0.586420,0.530864,0.592593,...,0.555556,0.425926,0.598765,0.438272,0.481481,0.574074,0.574074,0.666667,0.518519,0.400000
CHC,0.552147,0.413580,0.401235,0.543210,0.413580,0.543210,0.549383,0.487654,0.407407,0.524691,...,0.438272,0.376543,0.407407,0.450617,0.598765,0.635802,0.567901,0.582822,0.518519,0.566667
CHW,0.490798,0.462963,0.586420,0.512346,0.500000,0.530864,0.512346,0.611111,0.555556,0.444444,...,0.487654,0.524691,0.388889,0.450617,0.469136,0.481481,0.413580,0.382716,0.447205,0.583333
CIN,0.475309,0.588957,0.521472,0.407407,0.481481,0.425926,0.469136,0.447853,0.493827,0.444444,...,0.487654,0.598765,0.555556,0.469136,0.395062,0.419753,0.419753,0.413580,0.462963,0.516667
CLE,0.549383,0.598765,0.555556,0.561728,0.456790,0.419753,0.493827,0.574074,0.481481,0.592593,...,0.493827,0.419753,0.567901,0.524691,0.503106,0.583851,0.629630,0.561728,0.574074,0.583333


______________________________________________